#  XGBoost (eXtreme Gradient Boosting)
- XGBoost, GBM'in hız ve tahmin performansını arttırmak üzere optimize edilmiş; ölçeklenebilir ve farklı platformlara entegre edilebilir halidir.
- R, Python, Hadoop, Scala, Julia ile kullanılabilir.
- Öleçklenebilirdir.
- Hızlıdır.
- Tahmin başarısı yüksektir.
- Birçok Kaggle yarışmasında başarısı kanıtlanmıştır.


**Gerekli Kütüphaneler** 

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn import  neighbors
from sklearn.svm import SVR

In [39]:
# Uyarı Mesajları ile Karşılaşmamak için bu kütüphaneyi kullanıyoruz.
from warnings import filterwarnings
filterwarnings("ignore")

# XGBoost (eXtreme Gradient Boosting) - Model ve Tahmin İşlemleri 

In [40]:
df = pd.read_csv("./Hitters.csv")
# bu csv dosyasının içerisinde eksik gözlemleri(NA) çıkardık.
df = df.dropna()

# Veri seti içerisindeki kategorik değişkenleri dummy değişkenlere çeviriyoruz.
dms = pd.get_dummies(df[["League","Division","NewLeague"]])

# bağımlı değişken
y = df[["Salary"]]

# Veri Setinin içerisinden Bağımlı Değişkeni ve Kategorik değişkenlerin ilk hallerini dışarı bırakıyoruz.
X_ = df.drop(["Salary","League","Division","NewLeague"], axis = 1).astype("float64")

# dms ile X_ birleştirip(concat) bağımsız değişken oluşturduk.
X = pd.concat([X_, dms[["League_N","Division_W","NewLeague_N"]]], axis=1)

# train ve test setlerimizi oluşturuyoruz.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.25,
                                                    random_state= 42) 

In [41]:
import xgboost
from xgboost import XGBRegressor

**Model**

In [42]:
xgb_model = XGBRegressor().fit(X_train, y_train)

In [43]:
xgb_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

**Tahmin**

In [46]:
y_pred = xgb_model.predict(X_test)

In [47]:
np.sqrt(mean_squared_error(y_test,y_pred))

355.46515176059927

## Model Tuning

In [48]:
xgb_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth":[2,3,4,5],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree":[0.4,0.7,1]}

In [49]:
xgb_cv_model = GridSearchCV(xgb_model,xgb_params,
                            cv=10, 
                            n_jobs=-1,
                            verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   53.6s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  2.4min finished


In [50]:
xgb_cv_model.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 2,
 'n_estimators': 1000}

In [54]:
xgb_tuned = XGBRegressor(colsample_bytree = 0.4, learning_rate=0.1, max_depth = 2, n_estimators=1000).fit(X_train,y_train)

In [55]:
y_tuned_pred = xgb_tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_tuned_pred))

367.8515299923177